<a href="https://colab.research.google.com/github/marcomag416/MLDL/blob/main/BiSeNet_metrics_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation code

In [1]:
#install fvcore
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 330.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=d6bebd2ee9a3df481307205f46369bcf5eda857e49d5d980937eb1f48e8841bd
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=69c59dca0b98ce52bd979bc173efc5aebf5d01d6c1b0f1cfeb655bf083e33609
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [2]:
#download bisenet model from official repository
import sys
import requests
from zipfile import ZipFile
from io import BytesIO
model_url = "https://github.com/ooooverflow/BiSeNet/archive/refs/heads/master.zip"

# Send a GET request to the URL
response = requests.get(model_url)
# Check if the request was successful
if response.status_code == 200:
    #print(response.content)
    # Open the downloaded bytes and extract them
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('./')
    print('Download and extraction complete!')

sys.path.insert(0, './BiSeNet-master')

Download and extraction complete!


# Metrics computation
### FLOPS and parameters

In [3]:
#compute number of flops
from fvcore.nn import FlopCountAnalysis, flop_count_table

from model.build_BiSeNet import BiSeNet
from torch import nn
import torch
from torch.optim.lr_scheduler import PolynomialLR
import numpy as np
import os

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------
# Initizialize your model here

# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

context_path = 'resnet18'

model = BiSeNet(num_classes=19, context_path=context_path).to(device)
# -----------------------------

height = 512
width = 1024
image = torch.randn((1, 3, height, width)).to(device)

flops = FlopCountAnalysis(model, image)
print(flop_count_table(flops))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 71.6MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 165MB/s]


| module                                      | #parameters or shape   | #flops     |
|:--------------------------------------------|:-----------------------|:-----------|
| model                                       | 12.582M                | 26.101G    |
|  saptial_path                               |  0.371M                |  5.132G    |
|   saptial_path.convblock1                   |   1.856K               |   0.268G   |
|    saptial_path.convblock1.conv1            |    1.728K              |    0.226G  |
|    saptial_path.convblock1.bn               |    0.128K              |    41.943M |
|   saptial_path.convblock2                   |   73.984K              |   2.437G   |
|    saptial_path.convblock2.conv1            |    73.728K             |    2.416G  |
|    saptial_path.convblock2.bn               |    0.256K              |    20.972M |
|   saptial_path.convblock3                   |   0.295M               |   2.426G   |
|    saptial_path.convblock3.conv1            |    0.2

### Latency and FPS

In [4]:
import time
from tqdm.auto import tqdm

print(f"Using {device} device")

iters = 1000
latency = [0]*iters

for i in tqdm(range(iters)):
  start = time.time()
  out = model(image)
  end = time.time()

  latency[i] = end-start

latency = np.array(latency)
fps = 1 / latency

print(f"Mean latency: {latency.mean()} s \nStd latency: {latency.std()} s \nMean FPS: {fps.mean()}\nStd FPS: {fps.std()}")

Using cuda device


  0%|          | 0/1000 [00:00<?, ?it/s]

Mean latency: 0.02479734706878662 s 
Std latency: 0.006650048572157622 s 
Mean FPS: 41.98450034086914
Std FPS: 6.679459208724361
